# Práctica Final: Asistente Turístico de Tenerife (RAG)

Notebook de desarrollo paso a paso.

## 1. Configuración Inicial
Verificación de que el entorno y las claves API están correctamente cargadas.

In [1]:
import os
import sys
from dotenv import load_dotenv

# 1. Configuramos el path para poder importar módulos de src/
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

# 2. Cargamos variables de entorno
load_dotenv(os.path.join(project_root, '.env'))

# 3. Verificación
print(f"Directorio Raíz: {project_root}")

api_key = os.getenv("OPENAI_API_KEY")
if api_key and api_key.startswith("sk-"):
    print("✅ API Key de OpenAI cargada correctamente.")
else:
    print("❌ ERROR: No se detectó la API Key o el formato es incorrecto.")

Directorio Raíz: f:\development\Development\Master IA\LLM-large-language-models-entrega-final
✅ API Key de OpenAI cargada correctamente.


In [2]:
from langchain_openai import ChatOpenAI

print("⏳ Probando conexión con OpenAI...")
try:
    # Instanciamos el modelo básico
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
    response = llm.invoke("Di 'Conexión exitosa' si me lees.")
    print(f"✅ Respuesta del LLM: {response.content}")
except Exception as e:
    print(f"❌ Error de conexión con OpenAI: {e}")

⏳ Probando conexión con OpenAI...
✅ Respuesta del LLM: Conexión exitosa. ¿En qué puedo ayudarte hoy?


## 2. Ingesta de Datos (Lectura PDF)
Cargaremos la guía turística desde la carpeta `data/raw` y procesaremos su contenido.

In [3]:
from langchain_community.document_loaders import PyPDFLoader

# Ruta al archivo PDF (Ahora en data/raw)
pdf_path = os.path.join(project_root, "data", "raw", "TENERIFE.pdf")

print(f"📄 Buscando PDF en: {pdf_path}")

if os.path.exists(pdf_path):
    loader = PyPDFLoader(pdf_path)
    docs = loader.load()
    print(f"✅ PDF Cargado con éxito. Total páginas: {len(docs)}")
    
    # Verificamos contenido de una página al azar (pág 10, índice 9)
    print("\n--- Muestra de contenido (Pág 10) ---")
    print(docs[9].page_content[:500] + "...")
else:
    print("❌ ERROR: El archivo PDF no existe en la ruta especificada.")

📄 Buscando PDF en: f:\development\Development\Master IA\LLM-large-language-models-entrega-final\data\raw\TENERIFE.pdf
✅ PDF Cargado con éxito. Total páginas: 25

--- Muestra de contenido (Pág 10) ---
o La Playa de Los Patos 
o Playa del Ancón 
 
Nota: Para llegar estas playas hay que caminar un rato y algunas son de difícil acceso 
(la de Los Patos). También hay que ir a estas playas cuando la marea esté baja, si no...


In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Dividimos el texto en fragmentos (chunks) para que entren en el contexto del LLM
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,    # Caracteres por chunk
    chunk_overlap=200,  # Solapamiento para no perder contexto entre cortes
    length_function=len,
    add_start_index=True,
)

chunks = text_splitter.split_documents(docs)
print(f"✅ Documento dividido en {len(chunks)} fragmentos (chunks).")
print(f"Tamaño promedio del chunk: {sum(len(c.page_content) for c in chunks) / len(chunks):.0f} caracteres.")

✅ Documento dividido en 30 fragmentos (chunks).
Tamaño promedio del chunk: 570 caracteres.


## 3. Validación del Módulo `src.data.loader`
Ahora verificamos que el código refactorizado en `src/` funciona igual que el código experimental de arriba.

In [5]:
from src.data.loader import DataLoader

print("🔄 Probando clase DataLoader refactorizada...")
# Inicializamos el loader con la misma ruta
loader = DataLoader(pdf_path)

# Documentos
docs_refactored = loader.load()

# Chunks
chunks_refactored = loader.split(docs_refactored)

print(f"\n✅ Validación completada: {len(chunks_refactored)} chunks generados mediante la clase encapsulada.")

🔄 Probando clase DataLoader refactorizada...
📄 Cargando PDF desde: f:\development\Development\Master IA\LLM-large-language-models-entrega-final\data\raw\TENERIFE.pdf
✅ PDF cargado con 25 páginas.
✂️  Dividiendo 25 documentos en chunks...
✅ Se generaron 30 fragmentos.

✅ Validación completada: 30 chunks generados mediante la clase encapsulada.


## 4. Motor Vectorial (Embeddings)
Convertiremos los chunks de texto en vectores numéricos y los almacenaremos en ChromaDB.

In [6]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
import shutil

# Ruta donde guardaremos la base de datos vectorial
chroma_path = os.path.join(project_root, "chroma_db")

# Limpiamos la BD anterior si existe para empezar de cero
if os.path.exists(chroma_path):
    print("🧹 Limpiando base de datos vectorial existente...")
    shutil.rmtree(chroma_path)

print("🧠 Generando embeddings... (Esto puede tardar un poco)")

# Inicializamos Embeddings y VectorStore
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vector_store = Chroma.from_documents(
    documents=chunks_refactored,
    embedding=embeddings,
    persist_directory=chroma_path
)

print(f"✅ Base de datos vectorial creada en: {chroma_path}")

🧹 Limpiando base de datos vectorial existente...
🧠 Generando embeddings... (Esto puede tardar un poco)
✅ Base de datos vectorial creada en: f:\development\Development\Master IA\LLM-large-language-models-entrega-final\chroma_db


In [7]:
# Prueba de búsqueda semántica
query = "¿Qué altura tiene el Teide?"
print(f"🔍 Buscando: '{query}'")

results = vector_store.similarity_search(query, k=3)

for i, res in enumerate(results):
    print(f"\n--- Resultado {i+1} ---")
    print(res.page_content[:300] + "...")

🔍 Buscando: '¿Qué altura tiene el Teide?'

--- Resultado 1 ---
Si queréis subir hasta el pico del Teide, podéis hacerlo desde aquí haciendo uso de 
los teleféricos del Teide. Y, si queréis más info sobre el Teide, podéis ir al Centro de 
Visitantes de El Portillo que es gratis. 
Si os apetece, podéis subir de noche cuando esté despejado, ya que desde El Teide 
...

--- Resultado 2 ---
• El Teide 
Si vais a Tenerife y no subís al Parque Nacional del Teide, simplemente no habéis 
estado en Tenerife. 
 
Mi recomendación – esto es simplemente gustos personales míos – para subir al 
Teide es subir por la carretera TF24 (carretera de La Esperanza) desde la rotonda de 
Padre Anchieta en...

--- Resultado 3 ---
del Teide pertenece a La Orotava. 
o En las fiestas de La Orotava (consideradas Fiestas de Interés Turístico 
Nacional) se hace una alfombra gigante de arenas y tierras procedentes del 
Teide y tiene el récord Guinness de mayor tapiz de tierra del mundo. 
A La Orotava pertenecen tres pl

## 5. Validación del Módulo `src.data.vector_store`
Verificamos que la clase `VectorStoreManager` gestiona correctamente la creación y búsqueda.

In [8]:
from src.data.vector_store import VectorStoreManager

print("🔄 Probando VectorStoreManager refactorizada...")

chroma_path_ref = os.path.join(project_root, "chroma_db_refactored")

manager = VectorStoreManager(persist_directory=chroma_path_ref)

# Crear DB
manager.create_vector_store(chunks_refactored, reset=True)

# Buscar
results_ref = manager.search("¿Dónde se encuentra el auditorio Adán Martín?")

for i, res in enumerate(results_ref):
    print(f"\n--- [Refactor] Resultado {i+1} ---")
    print(res.page_content[:300] + "...")

🔄 Probando VectorStoreManager refactorizada...
🧹 Eliminando base de datos antigua en: f:\development\Development\Master IA\LLM-large-language-models-entrega-final\chroma_db_refactored
🧠 Generando embeddings y almacenando en ChromaDB...
✅ Base de datos vectorial lista en: f:\development\Development\Master IA\LLM-large-language-models-entrega-final\chroma_db_refactored
🔍 Buscando: '¿Dónde se encuentra el auditorio Adán Martín?'

--- [Refactor] Resultado 1 ---
o Auditorio de Tenerife [vídeo - ubicación] 
 
 
o Plaza de España [vídeo - ubicación]...

--- [Refactor] Resultado 2 ---
o Callejear dirección Plaza del Adelantado [ubicación] y tomar algo por ahí si 
hace buen tiempo (si queréis comer, os recomiendo La Cueva de Casiano - 
ubicación – y pedir la batata con almogrote y, de resto, que os recomienden 
ellos mismos). 
 
Los sitios de interés comentados son los siguientes:...

--- [Refactor] Resultado 3 ---
o Al ir hacia Buenavista, aparcar cerca de la Plaza de los Remedios y ver un 
po

## 6. Lógica RAG (Retrieval Augmented Generation)
Implementamos la cadena completa: Pregunta -> Búsqueda de Contexto -> Prompt con Contexto -> Respuesta LLM.

In [9]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 1. Definimos el Prompt del Asistente
template = """
Eres un guía turístico experto en Tenerife, amigable y conocedor.
Usa la siguiente información de contexto para responder a la pregunta del turista.
Si no encuentras la respuesta en el contexto, di amablemente que no tienes esa información, no inventes nada.

Contexto:
{context}

Pregunta del Turista: {question}

Respuesta:
"""
prompt = ChatPromptTemplate.from_template(template)

# 2. Obtenemos el Retriever (Buscador) de nuestra clase VectorStoreManager
# Usaremos 'manager' que validamos en el paso anterior y que ya tiene los datos cargados
retriever = manager.get_retriever(k=4)

# 3. Definimos la Cadena RAG (Chain)
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm  # Usamos el 'llm' que instanciamos al pricipio (ChatOpenAI)
    | StrOutputParser()
)

print("✅ Cadena RAG construida exitosamente.")

✅ Cadena RAG construida exitosamente.


In [10]:
# Prueba Real
pregunta = "¿Qué planes culturales recomiendas en Santa Cruz?"

print(f"👤 Pregunta: {pregunta}\n")
print("🤖 Generando respuesta...\n")

respuesta = rag_chain.invoke(pregunta)

print("💬 Respuesta del Asistente:")
print(respuesta)

👤 Pregunta: ¿Qué planes culturales recomiendas en Santa Cruz?

🤖 Generando respuesta...

💬 Respuesta del Asistente:
En Santa Cruz de Tenerife, te recomendaría visitar el Auditorio de Tenerife, que es un impresionante edificio de arquitectura moderna y uno de los símbolos de la ciudad. También puedes dar un paseo por la Plaza de España y disfrutar de la arquitectura colonial de la zona. Otra opción es visitar el Parque García Sanabria, un hermoso parque con esculturas y jardines. Además, puedes explorar el centro de la ciudad y descubrir sus tiendas, restaurantes y galerías de arte. ¡Espero que disfrutes de tu visita cultural en Santa Cruz!
